In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import pandas as pd
import subprocess as sp
import itertools as it

In [ ]:
serial_dir = "../Serial/data.csv"
parallel_seq_dir = "../ParallelSeq/data.csv"
parallel_grid_dir = "../ParallelGrid/data.csv"

In [ ]:
serial_data = pd.read_csv(serial_dir)
parallel_seq_data = pd.read_csv(parallel_seq_dir)
parallel_grid_data = pd.read_csv(parallel_grid_dir)